In [1]:
from chebGreen.greenlearning.utils import DataProcessor
from chebGreen.greenlearning.model import *
import matplotlib.pyplot as plt

# %env TF_MLC_LOGGING=1
# %env TF_CPP_MIN_LOG_LEVEL=2

In [2]:
example = 'laplace'
script = 'generate_example'

In [3]:
import os
matlab_path = "/Applications/MATLAB_R2023a.app/bin/matlab"
examplematlab = "\'"+example+"\'"
matlabcmd = f"{matlab_path} -nodisplay -nosplash -nodesktop -r \"{script}({examplematlab}); exit;\" | tail -n +11"
with open("temp.sh", 'w') as f:
    f.write(matlabcmd)
    f.close()
os.system(f"bash temp.sh")
os.remove("temp.sh")

### Example = laplace ###
Step = 1/100
Step = 2/100
Step = 3/100
Step = 4/100
Step = 5/100
Step = 6/100
Step = 7/100
Step = 8/100
Step = 9/100
Step = 10/100
Step = 11/100
Step = 12/100
Step = 13/100
Step = 14/100
Step = 15/100
Step = 16/100
Step = 17/100
Step = 18/100
Step = 19/100
Step = 20/100
Step = 21/100
Step = 22/100
Step = 23/100
Step = 24/100
Step = 25/100
Step = 26/100
Step = 27/100
Step = 28/100
Step = 29/100
Step = 30/100
Step = 31/100
Step = 32/100
Step = 33/100
Step = 34/100
Step = 35/100
Step = 36/100
Step = 37/100
Step = 38/100
Step = 39/100
Step = 40/100
Step = 41/100
Step = 42/100
Step = 43/100
Step = 44/100
Step = 45/100
Step = 46/100
Step = 47/100
Step = 48/100
Step = 49/100
Step = 50/100
Step = 51/100
Step = 52/100
Step = 53/100
Step = 54/100
Step = 55/100
Step = 56/100
Step = 57/100
Step = 58/100
Step = 59/100
Step = 60/100
Step = 61/100
Step = 62/100
Step = 63/100
Step = 64/100
Step = 65/100
Step = 66/100
Step = 67/100
Step = 68/100
Step = 69/100
Step = 70/100
Ste

In [4]:
data = DataProcessor("datasets/laplace/data.mat")
data.generateDataset(trainRatio = 0.95, batch_size = 1024)

In [5]:
model = GreenNN()
model.build()

In [ ]:
lossHistory = model.train(data, epochs = {'adam':int(2000), 'lbfgs':int(200)})

Training with Adam:
Loss at epoch 100: Training = 2.261E-02, Validation = 2.207E-02
Loss at epoch 200: Training = 2.722E-03, Validation = 2.601E-03


In [ ]:
plt.figure()
plt.semilogy(range(len(lossHistory['training'])), lossHistory['training'], 'b', alpha = 0.5)
plt.semilogy(range(len(lossHistory['validation'])), lossHistory['validation'], 'r--', alpha = 0.5)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Training','Validation'])

In [ ]:
xF, xU = data.xF, data.xU
x, y = np.meshgrid(xU, xF)
G = model.evaluateG(x,y)

fig = plt.figure(figsize = (8,6))
plt.contourf(x,y,G, 50, cmap = 'turbo', vmin = np.min(G), vmax = np.max(G))
plt.colorbar()

In [ ]:
model.saveModels('savedModels/laplace')